In [21]:
#step1: Setup audio recorder(ffmpeg , pyaudio & portaudio) 
#ffmped allows video, audio and image processing
import logging
import speech_recognition as sr
from pydub import AudioSegment
from io import BytesIO

In [ ]:
#step 1b 
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
#level is parameter :info is min, format m time -> level kya info ,debug -> message 
def record_audio(file_path, timeout=20, phrase_time_limit=None):
    """
    Simplified function to record audio from the microphone and save it as an MP3 file.

    Args:
    file_path (str): Path to save the recorded audio file.
    timeout (int): Maximum time to wait for a phrase to start (in seconds).
    phrase_time_lfimit (int): Maximum time for the phrase to be recorded (in seconds).
    """
    recognizer = sr.Recognizer()
    #initializing used to store the audio collected from microphone
    try:
        with sr.Microphone() as source:
            logging.info("Adjusting for ambient noise...")
            recognizer.adjust_for_ambient_noise(source, duration=1)#background k noise ko ignore
            logging.info("Start speaking now...")
            
            # Record the audio
            audio_data = recognizer.listen(source, timeout=timeout, phrase_time_limit=phrase_time_limit)#timeout time tk wait if no noise then bnd ho jayega and pharse time tk audio record krega
            logging.info("Recording complete.")
            
            # Convert the recorded audio to an MP3 file
            wav_data = audio_data.get_wav_data()#convert the recorded audio in wave format
            audio_segment = AudioSegment.from_wav(BytesIO(wav_data))#conver wave into file type so that pydub can read
            audio_segment.export(file_path, format="mp3", bitrate="128k")#save the file in mp3 format
            
            logging.info(f"Audio saved to {file_path}")

    except Exception as e:
        logging.error(f"An error occurred: {e}")

audio_filepath="patient_voice_test_for_patient.mp3"
#record_audio(file_path=audio_filepath)
record_audio(audio_filepath)


2025-04-20 01:11:13,429 - INFO - Adjusting for ambient noise...
2025-04-20 01:11:14,431 - INFO - Start speaking now...
2025-04-20 01:11:19,520 - INFO - Recording complete.
2025-04-20 01:11:19,673 - INFO - Audio saved to patient_voice_test_for_patient.mp3


In [24]:
#step 2 :convert audio into text(so that audio can be used as text in brain)
import os
from groq import Groq
GROQ_API_KEY=os.environ.get("GROQ_API_KEY")
client=Groq(api_key=GROQ_API_KEY)
stt_model="whisper-large-v3-turbo"#speech to text model

In [25]:
audio_file=open(audio_filepath, "rb")#open as rb read binary
transcription=client.audio.transcriptions.create(
    model=stt_model,
    file=audio_file,
    language="en"
)

print(transcription.text)

2025-04-20 01:11:58,025 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/audio/transcriptions "HTTP/1.1 200 OK"


 This channel is here. Hi, mic testing, mic testing, mic testing.
